In [238]:
import numpy as np
import matplotlib.pyplot as plt
import glob

from gensim.utils import tokenize, deaccent, simple_preprocess
from collections import Counter
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 

from sklearn import preprocessing
from sklearn.metrics.pairwise import cosine_similarity

# Νευρωνικά Δίκτυα και word embeddings

### Περιεχόμενα

- Βασικές έννοιες
- Απόσταση και ομοιότητα
- Word embeddings 
- Ταξινόμηση κειμένων 


## Βασικές έννοιες

### Κωδικοποίηση κειμένων

In [2]:
#### dataset: imbd movie reviews

In [3]:
#!mkdir ./data
#!rm ./data/*
#!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz -O ./data/dataset.tar.gz
#!tar xfz  ./data/dataset.tar.gz 

#### load texts

In [4]:
positive_train = glob.glob("./data/aclImdb/train/pos/*.txt")
negative_train = glob.glob("./data/aclImdb/train/neg/*.txt")
#negative_train[0:5], positive_train[0:5] 

In [5]:
positive_test = glob.glob("./data/aclImdb/test/pos/*.txt")
negative_test = glob.glob("./data/aclImdb/test/neg/*.txt")
len(negative_test), len(positive_test)

(12500, 12500)

In [6]:
sample_pos = open(positive_train[23]).read()
print( sample_pos[0:400] )

print("==============")
sample_neg = open(negative_train[32]).read()
print(sample_neg[0:400])

To all the reviewers on this page, I would have to say this movie is worth seeing. So It was made in 1972, so what. The fashion in the movie was exactly the same fashion of its time. People who didn't study culture of the decades would think that this movie is a cheese ball. Compared to the modern series, `Left Behind,' (Which is made for our time right now) it does look cheezy. However, the only 
The movie confuses religious ethics and ideals so much that it fails to create coherent argument against the death penalty on any level. By presenting the lawful execution of a convicted murder as the catalyst for the apocalyptic end of mankind the movie elevates a parent killer to the status of martyr for Christ. Somehow, according to the plot, god is outraged that society has chosen to rid it's 


### Αναπαράσταση 

Θέλουμε να αναπαραστήσουμε τις λέξεις σε μορφή διανύσματος έτσι ώστε να μπορέσουμε να τις δώσουμε ως είσοδο σε ένα νευρωνικό δίκτυο.

Ένας τρόπος να γίνει αυτό είναι η κωδικοποίηση "1-hot-encoding"
Στην κωδικοποίηση αυτή, βρίσκουμε όλες τις λέξεις των κειμένων μας και χτίζουμε ένα λεξικό ($V$), μια λίστα για παράδειγμα με όλες τις λέξεις που εμφάνίζονται στα κείμενα. Κάθε λέξη στη λίστα αυτή (λεξικό) εμφανίζεται μόνο μία φορά. 


Έστω ότι το λεξικό μας έχει $|V|$ λέξεις. Για κάθε λέξη δημιουργούμε ένα διάνυσμα $|V|$ θέσεων, στο οποίο η θέση στην οποία αντιστοιχεί στη θέση της λέξης στη λίστα η τιμή είναι 1, μηδέν διαφορετικά. 


Αν δηλαδή το λεξικό μας έχει τις λέξεις ["και", "το", "να"], το διάνυσμα που αντιστοιχεί στη λέξη "και" είναι το [1,0,0], στη λέξη "το", το διάνυσμα [0,1,0] και στη λέξη "να" το διάνυσμα [0,0,1]


Στην πράξη, επειδή ο αριθμός των λέξεων που εμφανίζονται στα κείμενα μπορεί να γίνει πολύ μεγάλος, συνήθως αφαιρούμε λέξεις με λιγότερες απο k εμφανίσεις.

Επίσης, για λόγους απλότητας και για να κρατήσουμε τον αριθμό λέξεων μικρό, κάνουμε όλα τα κείμενα lower case και παραλείπουμε σημεία στίξης

In [7]:
def build_vocab( files  ):
    
    counter = Counter()
    for f in files:
        text = open(f).read().lower()
        tokens = list(tokenize(deaccent(text)))
        counter.update(tokens)
    return counter


input_files = positive_train + negative_train
vocab_freqs = build_vocab( input_files ).most_common(1000)

#### χτίσιμο λεξικού

In [317]:
vocab = [word for word,_ in vocab_freqs]
word2idx = {word:i for i,word in enumerate(vocab)}
idx2word = {i:word for i,word in enumerate(vocab)}

In [321]:
def get_vector( word2idx, word):
    vec = np.zeros(len(word2idx),dtype=np.int32)
    vec[word2idx[word]] = 1
    return vec

v1 = get_vector(word2idx, "the")
v2 = get_vector(word2idx, "of")

v2[0:10]
vocab[0:10]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'it', 'in', 'i']

### ομοιότητα διανυσμάτων

θα θέλαμε να μετρήσουμε πόσο "παρόμοια" είναι 2 διανύσματα και ιδανικά, παρόμοιες σημασιολογικά λέξεις να έχουν μικρή απόσταση σε σχέση με λέξεις που δεν έχουν σημασιολογική συνάφεια


Από τη γραμμική άλγεβρα και την αναλυτική γεωμετρία, ένας τρόπος να μετρηθεί η ομοιότητα 2 διανυσμάτων $u, v$ είναι το εσωτερικό τους γινόμενο

$v \cdot u$. Γενικότερα όμως, επειδή δε μας ενδιαφέρει το "μήκος" των διανυσμάτων αλλά το να δείχνουν προς την ίδια κατεύθυνση, στην πράξη χρησιμοποιούμε την ομοιότητα συνημιτόνου, η οποία ορίζεται ώς

$$ sim_{cos} =  \frac{ v \cdot u }{ ||v|| \cdot || u ||} 
\overset{\Delta}{=}  
\frac{ \sum_{i}^{|V|}{u_i * v_i}} { \sqrt{ \sum_i^{|V|}{u_i^2} } \sqrt{ \sum_i^{|V|}{v_i^2} } }  $$  



Στην περίπτωση όμως του 1-hot encoding, όλα τα πιθανά ζεύγη διανυσμάτων u, v με $ u \neq v$, έχουν απόσταση 1. 


Η απόσταση ορίζεται ως $d(u,v) = 1-sim_{cos}(u,v)$


###  Neural language models και word2vec


Ένα (στατιστικό) γλωσσικό μοντέλο είναι μια κατανομή ακολουθιών λέξεων. Μας ενδιαφέρει να μοντελοποιήσουμε την πιθανότητα 

$p( w_{k+1} = w | w_1, w_2, ..., w_{k})$

Οι λόγοι που θέλουμε μια τέτοια μοντελοποίηση στην πράξη:

- spell checking 
- αναγνώριση φωνής 
- autocomplete 
- ... 

Συνήθως, τέτοια μόντέλα υπολογίζονται σε (τεράστιες) συλλογές κειμένων. Στη φυσική γλώσσα όμως, είναι βέβαιο ότι στην πράξη θα εμφανιστεί μια ακολουθία λέξεων που δεν υπάρχει στη συλλογή μας και θέλουμε να αποφύγουμε το μοντέλο μας να δώσει μηδενική πιθανότητα σε μια ακολουθία λέξεων. Παραδοσιακά, τέτοια προβλήματα λύνονταν με κάποιας μορφής παρεμβολή/προσέγγιση.


#### Neural models 

Το 2003, ο Bengio πρότεινε η συνάρτηση $p(w)$ να υπολογίζεται με εκπαίδευση νευρωνικού δικτύου στη συλλογή κειμένων,
έτσι ώστε η εισαγωγή στο δίκτυο να είναι k διανύσματα συνεχόμενων λέξεων ($w_1,...,w_k$) και το δίκτυο να προσπαθεί να προβλέψει ως έξοδο τη λέξη $w_{k+1}$. Η τεχνική αυτή είχε πολύ καλά αποτελέσματα αλλά ήταν πολύ αργή στην εκπαίδευση του νευρωνικού. 


Fast forward, 2013, ο Mikolov προτείνει το ίδιο ουσιαστικά μοντέλο αλλά με κάποιες σημαντικές διαφοροποιήσεις/ευρεστικές μεθόδους για την επιτάχυνση της εκπαίδευσης αλλά και την αρχιτεκτονική. το μοντέλο αυτό ονομάστηκε word2vec και ήταν η αρχή μιας τεράστιας έκρηξης στην επεξεργασία φυσικής γλώσσας.


Στην πράξη, αν πάρουμε για παράδειγμα τα κείμενα του imbd, που μπορεί να είναι αρκετές δεκάδες χιλιάδες λέξεων, το word2vec μας δίνει πίσω μια $D$-διάστατη απεικόνηση κάθε μιας από τις λέξεις, με $D << |V|$



Αρχιτεκτονικές CBOW και Skip-Gram

![αρχιτεκτονικές](O2YeO.png)

## Παράδειγμα word2vec με τη βιβλιοθήκη gensim

In [236]:

class MyCorpus(object):
    """An interator that yields sentences (lists of str)."""
    
    def __init__(self, files):
        self.files = files
            
    def __iter__(self):
        for file in self.files:
            
            text = open( file ).read().lower()
            
            yield simple_preprocess(text)

In [237]:
sentences = MyCorpus(positive_train + negative_train)
for s in sentences:
    print(s)
    break

['for', 'movie', 'that', 'gets', 'no', 'respect', 'there', 'sure', 'are', 'lot', 'of', 'memorable', 'quotes', 'listed', 'for', 'this', 'gem', 'imagine', 'movie', 'where', 'joe', 'piscopo', 'is', 'actually', 'funny', 'maureen', 'stapleton', 'is', 'scene', 'stealer', 'the', 'moroni', 'character', 'is', 'an', 'absolute', 'scream', 'watch', 'for', 'alan', 'the', 'skipper', 'hale', 'jr', 'as', 'police', 'sgt']


In [184]:
sentences = MyCorpus(positive_train + negative_train)

### Build and train a model

In [198]:
model = Word2Vec( min_count=5, workers=5, size=200) 
model.build_vocab(sentences)

In [199]:
sentences = MyCorpus(positive_train + negative_train)
model.train(sentences, total_examples=model.corpus_count, epochs=model.epochs)

(21181404, 28265970)

In [208]:
model.wv.most_similar("awful") 

[('terrible', 0.8418352603912354),
 ('horrible', 0.8121166825294495),
 ('amazing', 0.7599561214447021),
 ('atrocious', 0.7265543341636658),
 ('dreadful', 0.7250735759735107),
 ('awesome', 0.7141908407211304),
 ('laughable', 0.6928317546844482),
 ('ridiculous', 0.6859292387962341),
 ('bad', 0.6758812665939331),
 ('abysmal', 0.6707140803337097)]

In [241]:


v1 = model.wv["awful"].reshape(1,-1)
v2 = model.wv["terrible"].reshape(1,-1)


cosine_similarity( v1, v2)[0][0], model.wv.most_similar("awful")[0]

(0.84183526, ('terrible', 0.8418352603912354))

In [209]:
model.wv.most_similar("superb")

[('terrific', 0.9072020649909973),
 ('fantastic', 0.8522029519081116),
 ('magnificent', 0.8338733315467834),
 ('flawless', 0.8325532674789429),
 ('outstanding', 0.8291945457458496),
 ('brilliant', 0.827364981174469),
 ('marvelous', 0.823667049407959),
 ('splendid', 0.8175074458122253),
 ('excellent', 0.8133594989776611),
 ('exceptional', 0.8094056844711304)]

In [322]:
model.wv["superb"]

array([ 0.3489405 , -0.4663679 ,  0.07514985, -0.11252509,  0.63954186,
        0.6424196 ,  0.31715065,  0.42646608, -0.3214422 , -0.82062745,
       -1.3028479 ,  0.6060892 ,  1.0367268 ,  0.19444737,  0.00637792,
       -0.3790396 , -0.11567144, -0.52008176, -0.56881994, -0.5277116 ,
        1.2998116 , -0.26179138, -0.35718074,  0.4069805 , -1.1577462 ,
        0.7011286 , -0.02364672, -0.40308866, -0.40020722, -1.6058701 ,
        0.1491558 ,  0.01351472, -1.8364967 ,  0.19110866, -0.5087051 ,
       -0.2547878 ,  0.2595665 , -0.28101715, -0.6030559 , -1.2741352 ,
        0.35360038,  0.06381977,  0.89256   , -1.0443642 ,  0.4820704 ,
       -0.523485  ,  0.07598894,  0.43283203, -0.05821151,  0.00934027,
       -0.26078805,  0.05516728,  0.07939304, -0.37620443,  0.52051574,
        0.7022746 , -0.16445833, -0.6009418 , -0.6301671 ,  0.7340323 ,
        0.4210765 ,  0.18928401, -0.63667876, -0.25318676, -0.21367818,
       -0.74385947, -0.13740134,  0.31397378,  0.6059355 , -0.52

In [210]:
model.save("./data/model_v1.0")

In [126]:

model = Word2Vec()
model.build_vocab(sentences)

In [11]:
model = Word2Vec.load("./data/model_v1.0")

### Εφαρμογή: IMDB reviews sentiment classification


In [211]:
D=200
nwords = 500

### Απλή προσέγγιση: Η αναπαράσταση του κάθε κειμένου είναι ο μέσος όρος των διανυσμάτων των λέξεων του κειμένου

In [212]:
def file_to_vector( text, model, D,  nwords=1000 ):
    words = simple_preprocess(open(text).read())[0:nwords]
    
    c = 0 
    v = np.zeros(D)
    for word in words:
        if word in model.wv:
            c +=1 
            v+= model.wv[word]
        
      
    return v/c
    

In [213]:
X_pos = np.zeros( (len(positive_train), D))
y_pos = np.ones( len(positive_train) )

for idx,f in enumerate(positive_train):
    X_pos[idx,:] = file_to_vector(f, model, D=D, nwords=nwords )

In [324]:
X_pos.shape, y_pos[0:10]

((12500, 200), array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]))

In [205]:
model.epochs

5

In [214]:
X_neg = np.zeros( (len(negative_train), D))
y_neg = np.zeros( len(negative_train) )

for idx,f in enumerate(negative_train):
    X_neg[idx,:] = file_to_vector(f, model, D=D, nwords=nwords )

In [325]:
X_neg.shape, y_neg[0:10]

((12500, 200), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

In [215]:
X = np.concatenate( (X_pos, X_neg) , axis=0)
y= np.concatenate(  (y_pos, y_neg) , axis=0)

In [329]:
#X[0,:]

In [243]:
X = preprocessing.scale(X, axis=0) # zero mean, unit variance for each vector


In [244]:
X_train, X_test, y_train , y_test = train_test_split( X, y, random_state =42)
X_train.shape, X_test.shape

((18750, 200), (6250, 200))

### Λογιστική παλινδρόμιση για την ταξινόμιση των κειμένων: $p(l=1|text) = \frac{1}{1+exp(-x^T \cdot w)}$  

In [282]:
clf = LogisticRegression(max_iter = 1500, random_state = 42,fit_intercept=True) 
clf.fit( X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [330]:
X_test.shape

(6250, 200)

In [283]:
preds = clf.predict(X_test)
accuracy_score(y_test, preds)

0.8216

In [284]:
## 82% ακρίβεια πρόβλεψης με ένα απλό παράδειγμα 

array([[0.98037745]])

In [ ]:
v1 = model.wv["fantastic"]
v1 = np.append( v1, 1)
w =  np.append( clf.coef_, clf.intercept_)

In [332]:
v1 = model.wv["fantastic"]
v1 = np.append( v1, 1)
w =  np.append( clf.coef_, clf.intercept_)

1/(1+np.exp(-v1.dot(w))), 

(0.9796366717675539,)

In [331]:
clf.predict_proba( [model.wv["fantastic"]])

array([[0.02036333, 0.97963667]])

In [ ]:
clf.predict_proba( [model.wv["fantastic"]])

## [refs]
- Neural language model, Bengio : http://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf
- word2vec, Mikolov: https://arxiv.org/abs/1301.3781
- gensim: https://radimrehurek.com/gensim/ 
- Neural embeddings for metaphor detection: https://arxiv.org/pdf/1902.03659.pdf 
